In [10]:
import rasterio
from rasterio.plot import show
from rasterio.merge import merge
from rasterio.plot import show
import rasterio.features
import rasterio.warp
import glob
import os
import rioxarray as xr
import pandas as pd
import geopandas as gpd
import numpy as np
from rasterio.enums import Resampling
from rasterio.crs import CRS



%matplotlib inline


# Landcover Data

### Importing and Exploring the Landcover Data

In [11]:
#region of interest
studyRegion = '/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/Data/GeoJSONS/DRC_Training_Polygon.geojson'
studyRegion = gpd.read_file(studyRegion)
print(studyRegion.head())

                                            geometry
0  POLYGON ((30.80566 -8.21149, 30.45410 -7.58033...


In [12]:
#landcover data
ESA_CCI = '/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/Data/ESA_CCI/TIF/ESA_CCI_LC_Map_2013.tif'
landcover = xr.open_rasterio(ESA_CCI)
print(landcover)


<xarray.DataArray 'lccs_class' (time: 1, y: 64800, x: 129600)>
[8398080000 values with dtype=uint8]
Coordinates:
  * time         (time) object 2013-01-01 00:00:00
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 90.0 89.99 89.99 ... -89.99 -89.99 -90.0 -90.0
    spatial_ref  int64 0
Attributes:
    ancillary_variables:  processed_flag current_pixel_state observation_coun...
    flag_colors:          #ffff64 #ffff64 #ffff00 #aaf0f0 #dcf064 #c8c864 #00...
    flag_meanings:        no_data cropland_rainfed cropland_rainfed_herbaceou...
    flag_values:          [  0.  10.  11.  12.  20.  30.  40.  50.  60.  61. ...
    long_name:            Land cover class defined in LCCS
    standard_name:        land_cover_lccs
    valid_max:            220
    valid_min:            1
    _FillValue:           0.0
    scale_factor:         1.0
    add_offset:           0.0


In [13]:
#explore statistics
print(landcover.rio.crs)
print(landcover.rio.nodata)
print(landcover.rio.bounds())
print(landcover.rio.width)
print(landcover.rio.height)
print(landcover.rio.crs.wkt)

EPSG:4326
0
(-180.0, -90.0000000000144, 180.0000000000288, 90.0)
129600
64800
GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]


In [14]:
#landcover.values

### Reproject the Landcover Data to DRC Cooridnate Projection

In [15]:
#define projection for DRC
crs_drc = CRS.from_string('EPSG:3341')


In [ ]:
landcover_drc_crs = landcover.rio.reproject(crs_drc)
#landcover_drc_crs.rio.crs

### Clip to StudyRegion

In [ ]:
drc_landcover = landcover.rio.clip(studyRegion)

# Satellite Data

### Importing and Exploring the Landcover Data


In [ ]:
l8 = ''
l8_data = xr.open_rasterio(l8)
print(l8_data)

In [ ]:
#explore statistics
print(l8_data.rio.crs)
print(l8_data.rio.nodata)
print(l8_data.rio.bounds())
print(l8_data.rio.width)
print(l8_data.rio.height)
print(l8_data.rio.crs.wkt)

### Reproject the Landcover Data to DRC Cooridnate Projection

In [ ]:
l8_data_drc_crs = l8_data.rio.reproject(crs_drc)


### Resample Landsat Data to match resolution of Landcover Data

In [ ]:
#resample satellite data to same resolution as landcover data 
def write_image(arr: np.array, save_path: os.PathLike, **raster_meta) -> None:
    """
    Write a Geotiff to disk with the given raster metadata.
    Convenience function that automatically sets permissions correctly.
    Args:
        arr (np.array): The data to write to disk in geotif format
        save_path (os.PathLike): The path to write to
    """

    with rasterio.open(save_path, "w", **raster_meta) as target:
        target.write(arr)
    save_path.chmod(0o664)  # Allow group workspace users access

def generate_downsample(
    file_path: ESA_CCI,
    downsample_factor: int = 10, #resampling to 300m
    resampling: Resampling = Resampling.bilinear,
    overwrite: bool = False,
) -> None:
    """
    Generate downsample of the raster file at `file_path` and save it in same folder.
    Saved file will have appendix `_downsample_{downsample_factor}x.tif`
    Args:
        file_path (os.PathLike): The path to the raster image todownsample
        downsample_factor (int, optional): The downsampling factor to use.
            Defaults to 2.
        resampling (Resampling, optional): The resampling algorithm to use.
            Defaults to Resampling.bilinear.
        overwrite (bool, optional): Iff True, any existing downsampling file with the
            same downsampling factor will be overwritten. Defaults to False.
    """

    save_path = ESA_CCI + f"_downsample_{downsample_factor}x.tif"
    
    with rasterio.open(file_path) as image:
        downsampled_image, transform = downsample_image(
            image,
            image.indexes,
            downsample_factor=downsample_factor,
            resampling=resampling,
        )
        nbands, height, width = downsampled_image.shape
        
        write_image(
        downsampled_image,
        save_path,
        driver="GTiff",
        height=height,
        width=width,
        count=nbands,
        dtype="float64",
        crs=image.crs,
        transform=transform,
        nodata=image.nodata,
    )
 


In [ ]:
generate_downsample(l8_data_drc_crs)

In [ ]:
l8_data = landcover.rio.clip(studyRegion)

In [ ]:
landcover.plot()